In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(1)
# Generating training data
x1 = np.random.uniform(7, size=100).round(2)
x2 = np.random.uniform(7, size=100).round(2)
x3 = np.random.uniform(-7, 0, size=100).round(2)
x4 = np.random.uniform(-7, 0, size=100).round(2)

dataset_0_0 = pd.DataFrame({'x1': x1[ : 50], 'x2': x2[ : 50], 'y': [0]*50})
dataset_0_1 = pd.DataFrame({'x1': x1[50 : ], 'x2': x4[ : 50], 'y': [1]*50})
dataset_1_0 = pd.DataFrame({'x1': x3[ : 50], 'x2': x2[50 : ], 'y': [1]*50})
dataset_1_1 = pd.DataFrame({'x1': x3[50 : ], 'x2': x4[50 : ], 'y': [0]*50})

train_data = pd.concat([dataset_0_0, dataset_0_1, dataset_1_0, dataset_1_1])
train_data

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.scatter(train_data['x1'][train_data['y'] == 0], train_data['x2'][train_data['y'] == 0], color = 'blue', label='Class 0')
ax.scatter(train_data['x1'][train_data['y'] == 1], train_data['x2'][train_data['y'] == 1], color = 'orange', label='Class 1')
ax.legend()
plt.show

In [ ]:
np.random.seed(20)
# Generating training data
x1 = np.random.uniform(7, size=100).round(2)
x2 = np.random.uniform(7, size=100).round(2)
x3 = np.random.uniform(-7, 0, size=100).round(2)
x4 = np.random.uniform(-7, 0, size=100).round(2)

dataset_0_0 = pd.DataFrame({'x1': x1[ : 50], 'x2': x2[ : 50], 'y1': [0]*50})
dataset_0_1 = pd.DataFrame({'x1': x1[50 : ], 'x2': x4[ : 50], 'y1': [1]*50})
dataset_1_0 = pd.DataFrame({'x1': x3[ : 50], 'x2': x2[50 : ], 'y1': [1]*50})
dataset_1_1 = pd.DataFrame({'x1': x3[50 : ], 'x2': x4[50 : ], 'y1': [0]*50})

test_data = pd.concat([dataset_0_0, dataset_0_1, dataset_1_0, dataset_1_1])

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.scatter(test_data['x1'][test_data['y1'] == 0], test_data['x2'][test_data['y1'] == 0], color = 'blue', label='Class 0')
ax.scatter(test_data['x1'][test_data['y1'] == 1], test_data['x2'][test_data['y1'] == 1], color = 'orange', label='Class 1')
ax.legend()
plt.show

In [ ]:
# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [ ]:
# Shuffle train and test data
train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

In [ ]:
# Define features and target variable
X_train = train_data[['x1', 'x2']].values.astype(np.float32)
y_train = train_data['y'].values.reshape(-1, 1).astype(np.float32)
X_test = test_data[['x1', 'x2']].values.astype(np.float32)
y_test = test_data['y1'].values.reshape(-1, 1).astype(np.float32)

In [ ]:
# Create DataLoader
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=200, shuffle=True)

## MLP

In [ ]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [ ]:
# Initialize the model, loss function, and optimizer
model = MLP(input_size=2, hidden_size=4)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.03)
model

In [ ]:
# baseline accuracy and loss
accuracy, train_loss = 0, 0
for inputs, labels in train_loader:
    outputs = model(inputs)
    predicted_labels = torch.where(outputs >= 0.5 , 1.0, 0.0)
    accuracy += torch.sum(predicted_labels == labels)
    loss = criterion(outputs, labels)
    train_loss += loss.item() 

print(f'Baseline accuracy {accuracy / len(train_loader.dataset)}, Baseline loss {train_loss/len(train_loader)}')

In [ ]:
# Training loop
epochs = 1000
train_losses = []
for epoch in range(epochs):
    train_loss = 0.0
    accuracy = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        predicted_labels = torch.where(outputs >= 0.5 , 1.0, 0.0)
        accuracy += torch.sum(predicted_labels == labels)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() 
    
    if epoch % 100 ==0:
        print(f'Epoch {epoch} Accuracy {accuracy/len(train_loader.dataset):.4f} loss {train_loss/len(train_loader):.4f}')

    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
total_train_loss = np.mean(train_losses)
print(f'Training loss: {total_train_loss:.4f}')

In [ ]:
# MLP on test_data
test_loss = 0.0
accuracy = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted_labels = torch.where(outputs >= 0.5 , 1.0, 0.0)
        accuracy += torch.sum(predicted_labels == labels)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
print(f'Test loss: {test_loss/len(test_loader):.4f}, accuracy: {accuracy/len(test_loader.dataset):.4f}')

In [ ]:
input = torch.tensor([0.48, -0.15])
output = model(input)
print(output)

In [ ]:
# Create meshgrid data for visualization
xx = torch.arange(-7.0, 8.0).clone().detach()
yy = torch.arange(-7.0, 8.0).clone().detach()
xx, yy = torch.meshgrid(xx, yy)

# Flatten meshgrid data
xx1 = xx.reshape(-1, 1)
yy1 = yy.reshape(-1, 1)

# Combine xx1 and yy1 to create input data
data = torch.hstack([xx1, yy1])

# Get predictions from the model
z = model(data)

# Threshold predictions to get binary values
z = torch.where(z > 0.5, 1, 0)

# Reshape z to match the meshgrid dimensions
z = z.reshape(15, 15)


#### Decision surface of MLP 

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.contourf(xx, yy, z, colors = ['skyblue', 'orange', 'orange', 'skyblue'], levels = 2)
ax.scatter(test_data['x1'][test_data['y1']==0], test_data['x2'][test_data['y1']==0], color = 'blue', label='Class 0')
ax.scatter(test_data['x1'][test_data['y1']==1], test_data['x2'][test_data['y1']==1], color = 'red', label='Class 1')
ax.legend()
plt.show

## MLP with L1 Regularisation

In [ ]:
def create_train_val_loader(train_data, i, block_size, batch_size_train=200, batch_size_val=40):
    start = i * block_size
    end = start + block_size

    val_dataset = train_data[start:end].reset_index(drop=True)
    X_val = val_dataset[['x1', 'x2']].values.astype(np.float32)
    y_val = val_dataset['y'].values.reshape(-1, 1).astype(np.float32)
    val_dataset = CustomDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size_val, shuffle=True)

    Xy_train = pd.concat([train_data[:start], train_data[end:]], axis=0).reset_index(drop=True)
    X_train = Xy_train[['x1', 'x2']].values.astype(np.float32)
    y_train = Xy_train['y'].values.reshape(-1, 1).astype(np.float32)
    train_dataset = CustomDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)

    return train_loader, val_loader

In [ ]:
# 5-fold cross validation
K = 5
epochs = 1000
block = len(train_data) // K
valid_loss_acc = {}

for learning_rate in [0.0001, 0.001, 0.01, 0.1, 1]:
    for regularization_coefficient in [0.0001, 0.001, 0.01, 0.1, 1]:
        k_cross_val_loss = []
        k_cross_val_acc = []
        for i in range(K):
            #  Splitting Train into Train and Validation Dataset
            train_loader, val_loader = create_train_val_loader(train_data, i,block)

            # Initialize MLP
            model = MLP(input_size=2, hidden_size=4)
            criterion = nn.BCELoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            # Training the model
            train_losses = []
            for epoch in range(epochs):
                train_loss = 0.0
                accuracy = 0
                for inputs, labels in train_loader:
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    predicted_labels = torch.where(outputs >= 0.5, 1.0, 0.0)
                    accuracy += torch.sum(predicted_labels == labels)
                    loss = criterion(outputs, labels)

                    sum = 0
                    for params in model.parameters():
                        params = params.view(-1)
                        sum += torch.sum(torch.abs(params))  # L1 regularization term

                    loss += regularization_coefficient * sum
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item() 
                
                if epoch % 100 == 0:
                    print(f'Epoch {epoch} Accuracy {accuracy/len(train_loader.dataset):.4f}')

                train_loss /= len(train_loader)
                train_losses.append(train_loss)
                
            total_train_loss = np.mean(train_losses)
            print(f'In {i+1} Cross validation,for {learning_rate}_{regularization_coefficient} Training loss: {total_train_loss:.4f}')

            # Validation
            val_loss = 0.0
            accuracy = 0
            with torch.no_grad():
                for inputs, labels in val_loader:    
                    outputs = model(inputs)
                    predicted_labels = torch.where(outputs >= 0.5, 1.0, 0.0)
                    accuracy += torch.sum(predicted_labels == labels)

                    loss = criterion(outputs, labels)
                    for params in model.parameters():
                        params = params.view(-1)
                        sum += torch.sum(torch.abs(params))  # L1 regularization term
                    loss += regularization_coefficient * sum

                    val_loss += loss.item()

            k_cross_val_loss.append(val_loss/len(val_loader))
            k_cross_val_acc.append(accuracy/len(val_loader.dataset))
            print(f'Validation loss: {val_loss/len(val_loader):.4f}, Validation accuracy: {accuracy/len(val_loader.dataset):.4f}')

        valid_loss_acc[f'{learning_rate}_{regularization_coefficient}_loss'] = np.mean(k_cross_val_loss)
        valid_loss_acc[f'{learning_rate}_{regularization_coefficient}_acc'] = np.mean(k_cross_val_acc)


In [ ]:
t = pd.DataFrame(valid_loss_acc, index = ['val']).T
t['val'] = t['val'].apply(lambda x: round(x, 2))
t

In [ ]:
best_params = t.idxmin()  # Find the hyperparameters that minimize the validation loss
best_loss = t.min()  # Get the minimum validation loss
best_accuracy = t.loc[best_params, 'val']  # Get the accuracy corresponding to the best parameters

learning_rate = best_params['val'].split('_')[0]
regularization_coefficient = best_params['val'].split('_')[1]
validation_loss = best_loss['val']
validation_accuracy = best_accuracy

print("Best Hyperparameters (Minimize Validation Loss):")
print("Learning Rate:", learning_rate)
print("Regularization coefficient:", regularization_coefficient)
print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)


In [ ]:
model = MLP(input_size=2, hidden_size=4)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# regularization_coefficient = 0.001

# training the model
epochs = 1000
train_losses = []
for epoch in range(epochs):
    train_loss = 0.0
    accuracy = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        sum = 0
        for params in model.parameters():
            params = params.view(-1)
            sum += torch.sum(torch.abs(params))

        loss += regularization_coefficient * sum
        loss.backward()
        optimizer.step()

        train_loss += loss.item() 
    
    if epoch % 100 ==0:
        print(f'Epoch {epoch} Accuracy {accuracy/len(train_loader.dataset):.4f} loss {train_loss/len(train_loader):.4f}')

    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
total_train_loss = np.mean(train_losses)
print(f'Training loss: {total_train_loss:.4f}')

        

In [ ]:
# X_test = test_data[['x1', 'x2']].values.astype(np.float32)
# y_test = test_data['y1'].values.reshape(-1, 1).astype(np.float32)
# test_dataset = CustomDataset(X_test, y_test)
# test_loader = DataLoader(test_dataset, batch_size=200, shuffle=True)

test_loss = 0.0
accuracy = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted_labels = torch.where(outputs >= 0.5 , 1.0, 0.0)
        accuracy += torch.sum(predicted_labels == labels)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
test_loss /= len(test_loader)
print(f'Test loss: {test_loss:.4f}, accuracy: {accuracy/len(test_loader.dataset):.4f}')

In [ ]:
# Generating a grid of points for evaluation
x_values = torch.tensor(torch.arange(-7.0, 8.0), requires_grad=False)
y_values = torch.tensor(torch.arange(-7.0, 8.0), requires_grad=False)
x_values, y_values = torch.meshgrid(x_values, y_values)
x_values = x_values.reshape(-1, 1)
y_values = y_values.reshape(-1, 1)
grid_data = torch.hstack([x_values, y_values])

# Evaluating the model on the grid
model_outputs = model(grid_data)

# Thresholding the model outputs
thresholded_outputs = torch.where(model_outputs > 0.5, 1, 0)

# Reshaping the thresholded outputs into a 2D grid
z = thresholded_outputs.reshape(15, 15)

#### Decision surface of MLP with L1 Regularisation

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.contourf(xx, yy, z, colors = ['skyblue', 'orange', 'orange', 'skyblue'], levels = 2)
ax.scatter(test_data['x1'][test_data['y1']==0], test_data['x2'][test_data['y1']==0], color = 'blue',label='Class 0')
ax.scatter(test_data['x1'][test_data['y1']==1], test_data['x2'][test_data['y1']==1], color = 'red', label='Class 1')
ax.legend()
plt.show

## MLP with L2 Regularisation

In [ ]:
# 5-fold cross validation
K = 5
epochs = 1000
block = len(train_data) // K
valid_loss_acc = {}

for learning_rate in [0.0001, 0.001, 0.01, 0.1, 1]:
    for regularization_coefficient in [0.0001, 0.001, 0.01, 0.1, 1]:
        k_cross_val_loss = []
        k_cross_val_acc = []
        for i in range(K):
            #  Splitting Train into Train and Validation Dataset
            train_loader, val_loader = create_train_val_loader(train_data, i, block)

            # Initialize MLP
            model = MLP(input_size=2, hidden_size=4)
            criterion = nn.BCELoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            # Training the model
            train_losses = []
            for epoch in range(epochs):
                train_loss = 0.0
                accuracy = 0
                for inputs, labels in train_loader:
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    predicted_labels = torch.where(outputs >= 0.5, 1.0, 0.0)
                    accuracy += torch.sum(predicted_labels == labels)
                    loss = criterion(outputs, labels)

                    sum = 0
                    for params in model.parameters():
                        sum += torch.sum(torch.square(params))  # L2 regularization term

                    loss += regularization_coefficient * sum
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item() 
                
                if epoch % 100 == 0:
                    print(f'Epoch {epoch} Accuracy {accuracy/len(train_loader.dataset):.4f}')

                train_loss /= len(train_loader)
                train_losses.append(train_loss)
                
            total_train_loss = np.mean(train_losses)
            print(f'In {i+1} Cross validation,for {learning_rate}_{regularization_coefficient} Training loss: {total_train_loss:.4f}')

            # Validation
            val_loss = 0.0
            accuracy = 0
            with torch.no_grad():
                for inputs, labels in val_loader:    
                    outputs = model(inputs)
                    predicted_labels = torch.where(outputs >= 0.5, 1.0, 0.0)
                    accuracy += torch.sum(predicted_labels == labels)

                    loss = criterion(outputs, labels)
                    for params in model.parameters():
                        sum += torch.sum(torch.square(params))  # L2 regularization term
                    loss += regularization_coefficient * sum

                    val_loss += loss.item()

            k_cross_val_loss.append(val_loss/len(val_loader))
            k_cross_val_acc.append(accuracy/len(val_loader.dataset))
            print(f'Validation loss: {val_loss/len(val_loader):.4f}, Validation accuracy: {accuracy/len(val_loader.dataset):.4f}')

        valid_loss_acc[f'{learning_rate}_{regularization_coefficient}_loss'] = np.mean(k_cross_val_loss)
        valid_loss_acc[f'{learning_rate}_{regularization_coefficient}_acc'] = np.mean(k_cross_val_acc)


In [ ]:
t = pd.DataFrame(valid_loss_acc, index = ['val']).T
t['val'] = t['val'].apply(lambda x: round(x, 2))
t

In [ ]:
best_params = t.idxmin()  # Find the hyperparameters that minimize the validation loss
best_loss = t.min()  # Get the minimum validation loss
best_accuracy = t.loc[best_params, 'val']  # Get the accuracy corresponding to the best parameters

learning_rate = best_params['val'].split('_')[0]
regularization_coefficient = best_params['val'].split('_')[1]
validation_loss = best_loss['val']
validation_accuracy = best_accuracy

print("Best Hyperparameters (Minimize Validation Loss):")
print("Learning Rate:", learning_rate)
print("Regularization coefficient:", regularization_coefficient)
print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)


In [ ]:
model = MLP(input_size=2, hidden_size=4)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.03)
regularization_coefficient = 0.001

# training the model
epochs =1000
train_losses = []
for epoch in range(epochs):
    train_loss = 0.0
    accuracy = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        sum = 0
        for params in model.parameters():
            sum += torch.sum(torch.square(params))

        loss += regularization_coefficient * sum
        loss.backward()
        optimizer.step()
        train_loss += loss.item() 
    
    if epoch % 100 ==0:
        print(f'Epoch {epoch} Accuracy {accuracy/len(train_loader.dataset):.4f} loss {train_loss/len(train_loader):.4f}')

    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
total_train_loss = np.mean(train_losses)
print(f'Training loss: {total_train_loss:.4f}')
        

In [ ]:
# X_test = test_data[['x1', 'x2']].values.astype(np.float32)
# y_test = test_data['y1'].values.reshape(-1, 1).astype(np.float32)
# test_dataset = CustomDataset(X_test, y_test)
# test_loader = DataLoader(test_dataset, batch_size=200, shuffle=True)

test_loss = 0.0
accuracy = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted_labels = torch.where(outputs >= 0.5 , 1.0, 0.0)
        accuracy += torch.sum(predicted_labels == labels)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
test_loss /= len(test_loader)
print(f'Test loss: {test_loss:.4f}, accuracy: {accuracy/len(test_loader.dataset):.4f}')

In [ ]:
# Generating a grid of points for evaluation
x_values = torch.tensor(torch.arange(-7.0, 8.0), requires_grad=False)
y_values = torch.tensor(torch.arange(-7.0, 8.0), requires_grad=False)
x_values, y_values = torch.meshgrid(x_values, y_values)
x_values = x_values.reshape(-1, 1)
y_values = y_values.reshape(-1, 1)
grid_data = torch.hstack([x_values, y_values])

# Evaluating the model on the grid
model_outputs = model(grid_data)

# Thresholding the model outputs
thresholded_outputs = torch.where(model_outputs > 0.5, 1, 0)

# Reshaping the thresholded outputs into a 2D grid
z = thresholded_outputs.reshape(15, 15)

#### Decision surface of MLP with L2 Regularisation

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.contourf(xx, yy, z, colors = ['skyblue', 'orange', 'orange', 'skyblue'], levels = 2)
ax.scatter(test_data['x1'][test_data['y1']==0], test_data['x2'][test_data['y1']==0], color = 'blue', label='Class 0')
ax.scatter(test_data['x1'][test_data['y1']==1], test_data['x2'][test_data['y1']==1], color = 'red', label='Class 1')
ax.legend()
plt.show

## Logistic Regression Model

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        logits = self.linear(x)
        return logits

In [ ]:
# Shuffle train and test data
train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

### With x1^2

In [ ]:
X_train = pd.concat([train_data['x1'], train_data['x2'], train_data['x1']**2], axis = 1)
X_train.columns = ['x1','x2','x1^2']
X_train = torch.tensor(X_train.values, dtype= torch.float32)
y_train = train_data['y']
y_train = torch.tensor(y_train.values, dtype= torch.float32)
# y_train

In [ ]:
reg = LogisticRegression(3)
optimizer = torch.optim.Adam(reg.parameters(), lr=0.01)
converged = False

prev_loss = 1e8

i = 0
while not converged:
    optimizer.zero_grad()
    logits = reg(X_train)
    logits = logits.reshape(-1)
    # print(logits)
    loss = F.binary_cross_entropy_with_logits(logits, y_train)
    loss.backward()
    optimizer.step()
    if i%10==0:
        print(i, loss.item())
    if np.abs(prev_loss - loss.item()) < 1e-5:
        converged = True
    prev_loss = loss.item() 
    i = i + 1


pred = F.sigmoid(reg(X_train))
pred = torch.where(pred > 0.5 , 1.0, 0.0)
y_train = y_train.reshape(-1, 1)
correct_pred = torch.sum(pred == y_train)
acc = correct_pred / len(y_train)
print(f'Training accuracy using Logistic Regression with sqaure of x1 {acc}')

In [ ]:
X_test = pd.concat([test_data['x1'], test_data['x2'], test_data['x1']**2], axis = 1)
X_test.columns = ['x1','x2','x1^2']
X_test = torch.tensor(X_test.values, dtype= torch.float32)
y_test = test_data['y1']
y_test = torch.tensor(y_test.values, dtype= torch.float32)


pred = F.sigmoid(reg(X_test))
pred = torch.where(pred > 0.5 , 1.0, 0.0)
y_test = y_test.reshape(-1, 1)
correct_pred = torch.sum(pred == y_test)
acc = correct_pred / len(y_test)
print(f'Testing accuracy using Logistic Regression with sqaure of x1 {acc}')

In [ ]:
xx1 = torch.tensor(torch.arange(-7.0,8.0), requires_grad = False)
xx2 = torch.tensor(torch.arange(-7.0,8.0), requires_grad = False)
xx1, xx2 = torch.meshgrid(xx1, xx2)
xx1 = xx1.reshape(-1, 1)
xx2 = xx2.reshape(-1, 1)
xx3 = xx1**2
# print(xx3)
data = torch.hstack([xx1, xx2, xx3 ])
z = reg(data)
z = torch.where(z > 0.5, 1, 0)
z = z.reshape(15,15)

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.contourf(xx, yy, z, colors = ['skyblue', 'orange', 'orange', 'skyblue'], levels = 2)
ax.scatter(test_data['x1'][test_data['y1']==0], test_data['x2'][test_data['y1']==0], color = 'blue')
ax.scatter(test_data['x1'][test_data['y1']==1], test_data['x2'][test_data['y1']==1], color = 'red')

### With x1x2

In [ ]:
X_train = pd.concat([train_data['x1'], train_data['x2'], train_data['x1']*train_data['x2']], axis = 1)
X_train.columns = ['x1','x2','x1x2']
X_train = torch.tensor(X_train.values, dtype= torch.float32)
y_train = train_data['y']
y_train = torch.tensor(y_train.values, dtype= torch.float32)

In [ ]:
reg = LogisticRegression(3)
opt = torch.optim.Adam(reg.parameters(), lr=0.01)
converged = False

prev_loss = 1e8

i = 0
while not converged:
    opt.zero_grad()
    logits = reg(X_train)
    logits = logits.reshape(-1)
    loss = F.binary_cross_entropy_with_logits(logits, y_train)
    loss.backward()
    opt.step()
    if i%10==0:
        print(i, loss.item())
    if np.abs(prev_loss - loss.item()) < 1e-5:
        converged = True
    prev_loss = loss.item() 
    i = i + 1

pred = F.sigmoid(reg(X_train))
pred = torch.where(pred > 0.5 , 1.0, 0.0)
y_train = y_train.reshape(-1, 1)
correct_pred = torch.sum(pred == y_train)
acc = correct_pred / len(y_train)
print(f'Training accuracy using Logistic Regression with x1x2 {acc}')

In [ ]:
X_test = pd.concat([test_data['x1'], test_data['x2'], test_data['x1']*test_data['x2']], axis = 1)
X_test.columns = ['x1','x2','x1x2']
X_test = torch.tensor(X_test.values, dtype= torch.float32)
y_test = test_data['y1']
y_test = torch.tensor(y_test.values, dtype= torch.float32)

pred = F.sigmoid(reg(X_test))
pred = torch.where(pred > 0.5 , 1.0, 0.0)
y_test = y_test.reshape(-1, 1)
correct_pred = torch.sum(pred == y_test)
acc = correct_pred / len(y_test)
print(f'Testing accuracy using Logistic Regression {acc}')

In [ ]:
xx1 = torch.tensor(torch.arange(-7.0,8.0), requires_grad = False)
xx2 = torch.tensor(torch.arange(-7.0,8.0), requires_grad = False)
xx1, xx2 = torch.meshgrid(xx1, xx2)
xx1 = xx1.reshape(-1, 1)
xx2 = xx2.reshape(-1, 1)
xx3 = xx1 * xx2
data = torch.hstack([xx1, xx2, xx3 ])
z = reg(data)
z = torch.where(z > 0.5, 1, 0)
z = z.reshape(15,15)

#### Decision surface of Logistic regression

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.contourf(xx, yy, z, colors = ['skyblue', 'orange', 'orange', 'skyblue'], levels = 2)
ax.scatter(test_data['x1'][test_data['y1']==0], test_data['x2'][test_data['y1']==0], color = 'blue')
ax.scatter(test_data['x1'][test_data['y1']==1], test_data['x2'][test_data['y1']==1], color = 'red')